# EVALUATION (OBTENTION OF METRICS) OF THE DIFFERENT MODELS USING SPACY-LLM


#Se hace el NER

Instalación de las librerías necesarias

In [5]:
!pip install -q spacy-llm
!pip install -q spacy[transformers,lookups]
!pip install -q confection
!pip install -q openai
!pip install -q spacy[llm]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 255.9/255.9 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.8/126.8 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.5/98.5 MB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 197.8/197.8 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.0/314.0 kB 21.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 74.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 73.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sentence-transformers 3.2.1 requires transformers<5.0.0,>=4.41.0, but you have transformers 4.36.2 which is incompatible.


In [6]:
!pip install --upgrade spacy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.9/60.9 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.1/29.1 MB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.5/19.5 MB 38.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.2/9.2 MB 53.9 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
  Attempting uninstall: blis
    Found existing installation: blis 0.7.11
    Uninstalling blis-0.7.11:
      Successfully uninstalled blis-0.7.11
  Attempting uninstall: thinc
    Found existing installation: thinc 8.2.5
    Uninstalling thinc-8.2.5:
      Successfully uninstalled thinc-8.2.5
  Attempting uninstall: spacy
    Found existing installation: spacy 3.7.5
    Uninstalling spacy-3.7.5:
      Successfully uninstalled spacy-3.7.5
ERROR: pip's d

Se importan spacy y openai

In [7]:
import spacy
import openai
from google.colab import userdata
import os
from spacy_llm.util import assemble
import json

/usr/local/lib/python3.10/dist-packages/numpy/_core/_dtype.py:106: FutureWarning: In the future `np.bool` will be defined as the corresponding NumPy scalar.
  if dtype.type == np.bool:
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(


Clave de la API de OpenAI como variable de entorno

In [8]:
os.environ["OPENAI_API_KEY"] = userdata.get('apiKey')

Función que toma las entidades de un informe concreto y las ubica en un diccionario.

In [9]:
def get_entities(nlp, report):
    doc = nlp(report)
    entities = []
    for ent in doc.ents:
        entity_info = {
            "label": ent.label_,
            "start": ent.start_char,  # Start position in the text
            "end": ent.end_char  # End position in the text
        }
        entities.append(entity_info)  # Append the entity information to the list
    return entities

Función que crea un json a partir de un diccionario

In [10]:
def create_json(entities, report_name):
  with open(f"{report_name}Entities4Metrics.json", "w") as outfile:
    json.dump(entities, outfile)

Función que procesa un informe

In [11]:
def process_report(report, report_name, nlp):
  entities = get_entities(nlp, report)
  return entities


#PREPARACIÓN DE LOS RESULTADOS PARA SU EVALUACIÓN


Función que procesa todos los informes en el json de entrada. Devuelve la lista de entidades reconocidas por el modelo para cada informe. La lista se proporciona en un formate apropiado para su posterior evaluación.

In [12]:
def process_input_json(file_name, nlp):
  dictionary = json.load(open(file_name))
  list_of_lists_of_entities = []
  list_of_texts = []
  for key, value in dictionary.items():
    list_of_lists_of_entities.append(process_report(value, key, nlp))
    list_of_texts.append(value)
  return list_of_lists_of_entities,list_of_texts

Función que crea el nlp


In [13]:
def create_nlp():
  nlp = assemble("config.cfg")
  return nlp

Función que toma una lista de entidades presentes en el informe en orden de aparición y las transforma en un formato apropiado para la evaluación del modelo.

In [ ]:
#Expected format: entities
'''
entities = [
    ("Names of Patients", "James Turner"),
    ("Patient's Age", "45"),
    ("Patient's Sex", "Male"),
    ("Patient's Height", "175 cm"),
    ("Patient's Weight", "88 kg"),
    ("Admission date", "13th August 2023"),
    ("Medical discharge date", "20th August 2023"),
    ("Diagnosis", "chronic obstructive pulmonary disease (COPD)"),
    ("Symptom", "persistent cough"),
    ("Symptom", "shortness of breath"),
    ("Medication and dosing", "Tiotropium, 18 mcg once daily"),
    ("Medication and dosing", "Salbutamol inhaler, 100 mcg as needed"),
    ("Tests", "pulmonary function test")
]
text = "**Clinical Report 1**  \nPatient Name: James Turner  \nAge: 45  \nSex: Male  \nHeight: 175 cm  \nWeight: 88 kg  \n\nDuring Mr. Turner's recent examination on 14th August 2023, he presented with persistent cough and shortness of breath. After conducting a pulmonary function test, he was diagnosed with chronic obstructive pulmonary disease (COPD). Treatment included the administration of Tiotropium, 18 mcg once daily, and Salbutamol inhaler, 100 mcg as needed. He was admitted on 13th August 2023 and discharged on 20th August 2023. Follow-up is scheduled in four weeks to assess response to treatment.  \n"
'''

In [14]:
def parse(text, entities):
  pos_entities = 0
  lista = []
  i = 0
  while i < len(text) and pos_entities < len(entities):
    nomEnt,string = entities[pos_entities]
    if(i+len(string) < len(text) and  text[i:i+len(string)] == string):
      lista.append({"start":i,"end":i+len(string),"label":nomEnt})
      pos_entities += 1
    i += 1
  return lista

Función que procesa el fichero json con las etiquetas verdaderas de los textos:

In [15]:
def process_input_true_labels_json(file_name, list_of_texts):
  dictionary = json.load(open(file_name))
  list_of_lists_of_entities = []
  i = 0
  for report_name in dictionary:
      lista = dictionary[report_name]
      list_of_lists_of_entities.append(parse(list_of_texts[i], lista))
      i = i+1
  return list_of_lists_of_entities

Función que utiliza en orden las funciones anteriores

In [16]:
def main():
  nlp = create_nlp()
  predicted_labels, list_of_texts = process_input_json("test_reports.json", nlp)
  ground_truth_labels = process_input_true_labels_json("true_labels.json", list_of_texts)
  metrics = get_metrics(predicted_labels, ground_truth_labels, nlp)
  with open("metrics.json", "w") as outfile:
    json.dump(metrics, outfile, indent = 4)


Program:

In [32]:
main()

ValueError: The examples file expects a .yml, .yaml, .json, or .jsonl file type. Ensure that your file corresponds to one of these file formats.

#Obtención Automatizada de Métricas
Referencia: https://rumn.medium.com/nervaluate-the-ultimate-way-for-benchmarking-ner-models-b29e83fbae95


In [18]:
!pip install nervaluate

In [19]:
from nervaluate import Evaluator
def get_metrics(predicted_labels, ground_truth_labels, nlp):
  evaluator = Evaluator(ground_truth_labels, predicted_labels, tags = list(nlp.get_pipe("llm").labels) , loader="default")

  # call the evaluator to get model evaluation result
  return (evaluator.evaluate())


#Extra: añadir automáticamente el fichero de configuración y el fichero con los informes

Se importa desde el repositorio de GitHub:

In [26]:
%cd /

/


In [2]:
!pip install -q requests

In [3]:
from google.colab import userdata
import os
import json

In [27]:
import requests
def download_file(url,local_filename):
  response = requests.get(url)
  with open(local_filename, 'wb') as file:
      file.write(response.content)

#descargar FICHEROS
url = "https://raw.githubusercontent.com/gonsan33/Concurso-ModelizaciOn-Empresas-24/main/GMV/CONFIG_FILES/config_GPT4.cfg"
local_filename = "content/config.cfg"
download_file(url,local_filename)

#url = "https://raw.githubusercontent.com/gonsan33/Concurso-ModelizaciOn-Empresas-24/main/GMV/TEST_REPORTS_AND_METRICS/true_labels.json"
url = "https://raw.githubusercontent.com/gonsan33/Concurso-ModelizaciOn-Empresas-24/main/GMV/TEST_REPORTS_AND_METRICS/long_medical_reports_true_labels.json"
local_filename = "content/true_labels.json"
download_file(url,local_filename)
#url = "https://raw.githubusercontent.com/gonsan33/Concurso-ModelizaciOn-Empresas-24/main/GMV/TEST_REPORTS_AND_METRICS/test_reports.json"
url = "https://raw.githubusercontent.com/gonsan33/Concurso-ModelizaciOn-Empresas-24/main/GMV/TEST_REPORTS_AND_METRICS/long_medical_reports.json"
local_filename = "content/test_reports.json"
download_file(url,local_filename)


Es necesario dejar la ruta como estaba en un principio

In [28]:
%cd content/

/content
